In [6]:
%pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from keras import layers
import tensorflow as tf
from pathlib import Path
import cv2 as cv

# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from keras.preprocessing.image import ImageDataGenerator

In [7]:
input_shape = (256, 256)
data_dir = "data/"

In [8]:
# Set input shape and number of classes
input_shape = (256, 256)
num_classes = 4

# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

# Load images and labels from directories
train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode="categorical",  # This ensures labels are one-hot encoded
    subset="training",
)

val_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode="categorical",
    subset="validation",
)

Found 8943 images belonging to 4 classes.
Found 2235 images belonging to 4 classes.


In [9]:
def create_train_save_model(save_path: str, epochs: int = 5) -> None:
    data_dir = "./data/"
    categories = ["Voiga", "Me", "Keanu", "Rest"]

    data_gen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

    train_data = data_gen.flow_from_directory(
        data_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode="categorical",
        subset="training",
    )
    val_data = data_gen.flow_from_directory(
        data_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode="categorical",
        subset="validation",
    )

    model = Sequential(
        [
            Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation="relu"),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation="relu"),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation="relu"),
            Dense(len(categories), activation="softmax"),
        ]
    )

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    model.fit(train_data, validation_data=val_data, epochs=epochs)

    model.save(save_path)

In [10]:
create_train_save_model("./saved_models/model_1_e1.keras", 1)
create_train_save_model("./saved_models/model_1_e5.keras", 5)

Found 8943 images belonging to 4 classes.
Found 2235 images belonging to 4 classes.


C:\Users\Olek Zima\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Olek Zima\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


280/280 ━━━━━━━━━━━━━━━━━━━━ 100s 346ms/step - accuracy: 0.9611 - loss: 0.1296 - val_accuracy: 0.9996 - val_loss: 0.0020
Found 8943 images belonging to 4 classes.
Found 2235 images belonging to 4 classes.
Epoch 1/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 97s 335ms/step - accuracy: 0.9735 - loss: 0.1143 - val_accuracy: 0.9996 - val_loss: 0.0080
Epoch 2/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 96s 336ms/step - accuracy: 0.9996 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss: 4.3940e-04
Epoch 3/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 96s 337ms/step - accuracy: 1.0000 - loss: 2.1331e-05 - val_accuracy: 1.0000 - val_loss: 5.7646e-05
Epoch 4/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 96s 335ms/step - accuracy: 0.9998 - loss: 5.5751e-04 - val_accuracy: 0.9996 - val_loss: 0.0028
Epoch 5/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 96s 338ms/step - accuracy: 0.9998 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 9.7747e-07


In [11]:
test_model_1 = tf.keras.models.load_model("./saved_models/model_1_e1.keras")
test_model_2 = tf.keras.models.load_model("./saved_models/model_1_e5.keras")

In [3]:
import cv2
import numpy as np

# Load the trained model
model = tf.keras.models.load_model("./saved_models/model_1_e1.keras")

# Load the face detection model
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Start the webcam feed
cap = cv2.VideoCapture(0)
class_names = ["Voiga", "Me", "Keanu", "Rest"]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_center_x = frame.shape[1] // 2
    img_center_y = frame.shape[0] // 2

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for x, y, w, h in faces:
        roi_color = frame[y : y + h, x : x + w]
        roi_color_resized = cv2.resize(
            roi_color, (64, 64)
        )
        roi_color_normalized = (
            roi_color_resized / 255.0
        )
        roi_color_expanded = np.expand_dims(
            roi_color_normalized, axis=0
        )
        predictions = model.predict(roi_color_expanded)
        predicted_class = np.argmax(predictions, axis=1)[0]

        if predicted_class >= len(class_names):
            label = "Rest"
        else:
            label = class_names[predicted_class]

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(
            frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2
        )

        face_center_x = x + w // 2
        face_center_y = y + h // 2

        cv2.line(
            frame,
            (img_center_x, img_center_y),
            (face_center_x, face_center_y),
            (0, 255, 0),
            2,
        )

        distance = np.sqrt(
            (face_center_x - img_center_x) ** 2 + (face_center_y - img_center_y) ** 2
        )
        cv2.putText(
            frame,
            f"Dist: {int(distance)}",
            (face_center_x, face_center_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1,
        )

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━

In [15]:
cv2.destroyAllWindows()